# Sequential writing

In [1]:
import bison
import numpy

Let's initialize an empty bison file in writing mode

In [2]:
f = bison.FILE('test', mode='w')

Let's write some data into our bison file. To do so we simply invoke the 'write' method of the 'FILE' class with an additional string, namely a unique tag, to label the content of the data

In [3]:
T = 96
Nc = 5

f.write('T',T)
f.write('ncorr', Nc)
f.write('desc', 'a string describing the data')

[Bison] : T with 6.67572e-06 MB written at 0.00735101 MB/s
[Bison] : ncorr with 9.53674e-06 MB written at 0.0123191 MB/s
[Bison] : desc with 3.62396e-05 MB written at 0.0784719 MB/s


We imagine to have a function which performs a calculation that takes long time

In [4]:
def test_func(n):
    return numpy.random.rand(96*n).reshape(n,96)

For each instance of the calculation we write to disk the results in the bison FILE

In [5]:
for i in [0,1,2,3,4]:
    dat = test_func(Nc)
    f.write(f'calculation number {i}', dat)

[Bison] : calculation number 0 with 0.00376701 MB written at 2.98226 MB/s
[Bison] : calculation number 1 with 0.00376701 MB written at 5.90654 MB/s
[Bison] : calculation number 2 with 0.00376701 MB written at 8.60098 MB/s
[Bison] : calculation number 3 with 0.00376701 MB written at 8.70044 MB/s
[Bison] : calculation number 4 with 0.00376701 MB written at 9.08046 MB/s


The content of the file can be easily imported by opening a new FILE instance in reading mode

In [6]:
ff = bison.FILE('test', mode='r')
res = ff.read()

[Bison] : Reading file test
[Bison] : File created by mbruno at macthxbruno.lan on Sun Dec 17 22:34:36 2023
[Bison] : Read 0.0190096 MB at 6.84865 MB/s


In [7]:
res['calculation number 4']

array([[0.80225737, 0.70375386, 0.86465048, 0.63461449, 0.31462978,
        0.28150328, 0.17274223, 0.89829944, 0.26021759, 0.92070715,
        0.93058145, 0.7709853 , 0.17563447, 0.52415459, 0.02000619,
        0.38554605, 0.52955647, 0.40984308, 0.09962975, 0.37436244,
        0.55467646, 0.19462026, 0.89962351, 0.3074037 , 0.75241438,
        0.56021382, 0.26279381, 0.14577777, 0.80576792, 0.81877911,
        0.57445305, 0.32583625, 0.23597202, 0.59568577, 0.84252923,
        0.67747952, 0.90982872, 0.37681215, 0.30209915, 0.73830714,
        0.39338582, 0.1447888 , 0.66735443, 0.72381351, 0.10759572,
        0.6670577 , 0.61345446, 0.8851517 , 0.43129965, 0.91745416,
        0.89422255, 0.70943251, 0.22503525, 0.95082019, 0.53515326,
        0.93862371, 0.79422065, 0.22322211, 0.89888103, 0.42680045,
        0.58888113, 0.53487978, 0.04345715, 0.89563877, 0.50112214,
        0.32274059, 0.46372345, 0.23870405, 0.9918967 , 0.53509232,
        0.42231985, 0.1677092 , 0.09398063, 0.47

At this point the bison FILE is still open, which means that no file has yet been created, but instead all data has been dumped into a temporary directory. Let's check it, we should find a `test.temp` directory

In [8]:
!ls -al | grep test

drwxr-xr-x   4 mbruno  staff    128 Dec 17 22:34 test.temp


Now we can close the file and therefore also eliminate the possibility to further append to it, or we can close the file as is, to a new destination, thus leaving the possibility to append more data in the future.

In [9]:
f.close(dest='test2', keep=True)
!ls -al | grep test

drwxr-xr-x   4 mbruno  staff    128 Dec 17 22:34 test.temp
-rw-r--r--   1 mbruno  staff  19937 Dec 17 22:34 test2


Now that a single file has been created, we can import it also with the standard loading command

In [10]:
res = bison.load('test2')
res.keys()

[Bison] : Reading file test2
[Bison] : File created by mbruno at macthxbruno.lan on Sun Dec 17 22:34:36 2023
[Bison] : Read 0.0190134 MB at 12.7536 MB/s


dict_keys(['T', 'ncorr', 'desc', 'calculation number 0', 'calculation number 1', 'calculation number 2', 'calculation number 3', 'calculation number 4'])

In [11]:
# final clean up
!rm -r test2

In [12]:
# test append mode
fff = bison.FILE('test', mode='a')
fff.write('T','new content here')

BisonError: Tag T already used

In [13]:
fff.write('new','new content here')
fff.close()
!ls -al | grep test

[Bison] : new with 2.38419e-05 MB written at 0.0198491 MB/s
-rw-r--r--   1 mbruno  staff  19964 Dec 17 22:34 test


In [14]:
!rm test